In [1]:
!pip install transformers

In [2]:
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.1 MB/s eta 0:00:00


In [3]:
!pip install loguru

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 1.7 MB/s eta 0:00:00


In [4]:
# This cell will authenticate you and mount your Drive in the Colab.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# %cd drive/MyDrive/projects/compositional-reasoning-finetuning
%cd drive/MyDrive/projects/compositional-reasoning-finetuning

/content/drive/MyDrive/projects/compositional-reasoning-finetuning


## Chunk training data

In [ ]:
import json

model = "self_ask_train-answer_first=False-random_facts=False"
in_file = f"data/FinetuningData/{model}.json"
out_dir = f"data/FinetuningData/{model}/"
with open(in_file, "r") as f:
  train = json.load(f)

chunks = 15
chunk_size = len(train) // chunks
start = 0
end = chunk_size
chunk_num = 1
while end <= len(train):
  with open(out_dir+f"chunk-{chunk_num}.json", "w") as f:
    json.dump(train[start: end], f)
  print(start, end)
  start += chunk_size
  end += chunk_size
  chunk_num+=1

0 7086
7086 14172
14172 21258
21258 28344
28344 35430
35430 42516
42516 49602
49602 56688
56688 63774
63774 70860
70860 77946
77946 85032
85032 92118
92118 99204
99204 106290


## Direct Finetuning

In [6]:
from training_utils import finetune_self_ask

In [7]:
# model_name
huggingface_model_name = 't5-small'
model = "direct-random_facts=False"

### ======= MAKE CHANGES HERE
# traing and validation file path
valid_file = 'data/FinetuningData/direct_dev-random_facts=False.json'
checkpoint_dir = f'checkpoints/{model}/'
prev_checkpoint = ""
# prev_checkpoint = f"{checkpoint_dir}/weights-{chunk_start-1}.hdf5"
# =============================

# hyper parameters
max_length = 300
batch_size = 32
epochs = 1

train_file = 'data/FinetuningData/direct_train-random_facts=False.json'

# path and file name for checkpoint
checkpoint_filepath = checkpoint_dir + f'weights' + '.{epoch:02d}-{batch:00005d}.hdf5'

model_wrapper = finetune_self_ask(
    huggingface_model_name,
    train_file,
    valid_file,
    checkpoint_filepath,
    max_length,
    batch_size,
    epochs,
    prev_checkpoint)

model_wrapper.save(f"models/{model}.h5", save_format="h5")

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


3321/3321 [==============================] - 1779s 526ms/step - loss: 0.0142 - accuracy: 0.9984 - val_loss: 8.1436e-04 - val_accuracy: 0.9998


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
